# 2.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot
* Chain-Of-Thought

In [1]:
from utils.pdf2prompts import pdf2prompts
import os
import asyncio
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain.schema.messages import AIMessage, HumanMessage

load_dotenv(find_dotenv())

model = ChatOpenAI(
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    model_name="gpt-4o",
    model_kwargs={"top_p": 0, "seed": 42},
    temperature=0,
)

In [2]:
pdf_path = "../data/raw/bpläne/2_zeichnung_textteil_getrennt/F 11- 02 Gewerbegebiet Himmelreich - schriftlicher Teil.pdf"
pdf_prompts = pdf2prompts(pdf_path)

In [14]:
# A) Ein Thread
msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Informationen aus einem Bebauungsplan zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
                {
                    "type": "text",
                    "text": 'Extrahiere Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
                    # Optional: Zusätzlich Dach, Garage und Stellplatz
                },
                *pdf_prompts,
            ]
        ),
    ])
print(msg.content)

Hier sind die extrahierten Informationen aus dem Bebauungsplan "Gewerbegebiet Himmelreich" zu den angefragten Themen:

### Art der baulichen Nutzung
- **Eingeschränktes Gewerbegebiet (GEE)** (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)
  - Zulässig sind Gewerbebetriebe, die das Wohnen nicht wesentlich stören, Einzelhandelsbetriebe mit bestimmten Sortimenten, und der Verkauf von auf dem Grundstück produzierten Waren auf untergeordneter Fläche.
  - Nicht zulässig sind Vergnügungsstätten und Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten.

- **Gewerbegebiet (GE)** (§ 8 BauNVO)
  - Zulässig sind Gewerbebetriebe aller Art, Einzelhandelsbetriebe mit bestimmten Sortimenten, und der Verkauf von auf dem Grundstück produzierten Waren auf untergeordneter Fläche.
  - Nicht zulässig sind Vergnügungsstätten und Einzelhandelsbetriebe mit innenstadtrelevanten und bestimmten nicht innenstadtrelevanten Sortimenten.

### Maß der baulichen Nutzung
- **Grundfläch

In [16]:
# B) Mehrere Threads
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def summarize(pdf_prompt):
    msg = model.invoke([
        AIMessage(
            content=[
                {
                    "type": "text",
                    "text": "Du bist ein Assistent um Textinformationen aus dem schriftlichen Teil eines Bebauungsplans zu extrahieren."
                },
            ]
        ),
        HumanMessage(
            content=[
            {
                "type": "text",
                "text": 'Extrahiere Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
            },
            pdf_prompt
            ]
        ),
    ])
    return msg.content

prompt_chain = list(map(lambda pdf_prompt: summarize(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)

for result in results:
    print(result)

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
- **Art der baulichen Nutzung**: Eingeschränktes Gewerbegebiet (GEE) (§ 8 BauNVO i.V.m. § 1 (4) BauNVO)

- **Maß der baulichen Nutzung**: -

- **Bauweise**: -

- **Überbaubare Grundstücksfläche**: -
Hier sind die extrahierten Informationen:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: 
  - Grundflächenzahl (§ 19 BauNVO): siehe Einträge im Lageplan
  - Geschossflächenzahl (§ 20 BauNVO): siehe Einträge im Lageplan
  - Höhe der baulichen Anlagen (§ 18 BauNVO): siehe Einträge im Lageplan. Die Gebäudehöhe wird gemessen von der Erdgeschossfußbodenhöhe (EFH) bis zur Schnittkante zwischen Außenwand und Dachhaut bzw. bis zur Oberkante Firstziegel.
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -
- Art der baulichen Nutzung: -

- Maß der baulichen Nutzung: -

- Bauweise: 
  - Abweichende Bauweise. Es gilt die offene Bauweise, jedoch sind die Gebäudelängen